<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context Managers and else Blocks


The semantics of for/else, while/else, and try/else are closely related, but very
different from if/else. Initially the word else actually hindered my understanding of
these features, but eventually I got used to it.
Here are the rules:

for

The else block will run only if and when the for loop runs to completion (i.e., not
if the for is aborted with a break).

while

The else block will run only if and when the while loop exits because the condition
became falsy (i.e., not when the while is aborted with a break).

try

The else block will only run if no exception is raised in the try block. The official
docs also state: “Exceptions in the else clause are not handled by the preceding
except clauses.”

In all cases, the else clause is also skipped if an exception or a return, break, or
continue statement causes control to jump out of the main block of the compound
statement.

Context manager objects exist to control a with statement, just like iterators exist to
control a for statement.

The with statement was designed to simplify the try/finally pattern, which guarantees
that some operation is performed after a block of code, even if the block is aborted
because of an exception, a return or sys.exit() call. The code in the finally clause
usually releases a critical resource or restores some previous state that was temporarily
changed.

The context manager protocol consists of the \_\_enter\_\_ and \_\_exit\_\_ methods. At the
start of the with, \_\_enter\_\_ is invoked on the context manager object. The role of the
finally clause is played by a call to \_\_exit\_\_ on the context manager object at the end
of the with block.

In [0]:
# Example 15-3. mirror.py: code for the LookingGlass context manager class

class LookingGlass:
  
  def __enter__(self):
    import sys
    self.original_write = sys.stdout.write   # Hold the original sys.stdout.write method in an instance attribute for later use.
    sys.stdout.write = self.reverse_write
    
    return 'JABBERWOCKY'
  
  def reverse_write(self, text):
    self.original_write(text[::-1])
    
  def __exit__(self, exc_type, exc_value, traceback):
    sys.stdout.write = self.original_write
    if exc_type is ZeroDivisionError:
      print('Please DO NOT divide by zero!')
      return True

## Using @contextmanager

This function is a decorator that can be used to define a factory function for with statement context managers, without needing to create a class or separate \_\_enter\_\_() and \_\_exit\_\_() methods.

While many objects natively support use in with statements, sometimes a resource needs to be managed that isn’t a context manager in its own right, and doesn’t implement a close() method for use with contextlib.


In a generator decorated with @contextmanager, yield is used to split the body of the
function in two parts: everything before the yield will be executed at the beginning of
the while block when the interpreter calls \_\_enter\_\_; the code after yield will run
when \_\_exit\_\_ is called at the end of the block

In [0]:
# example 15-5. mirror_gen.py: a context manager implemented with a generator
import contextlib
@contextlib.contextmanager
def looking_glass():
  import sys
  original_write = sys.stdout.write
  
  def reverse_write(text):              # Define custom reverse_write function; original_write will be available in the closure.
    original_write(text[::-1])
    
  sys.stdout.write = reverse_write
  yield 'JABBERWOCKY'
  sys.stdout.write = original_write

In [2]:
# Example 15-6. Test driving the looking_glass context manager function
with looking_glass() as what:
  print('Alice, Kitty and Snowdrop')
  print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


### Remark

Essentially the contextlib.contextmanager decorator wraps the function in a class
that implements the \_\_enter\_\_ and \_\_exit\_\_ methods.4
The \_\_enter\_\_ method of that class:
1. Invokes the generator function and holds on to the generator object—let’s call it gen.
2. Calls next(gen) to make it run to the yield keyword.
3. Returns the value yielded by next(gen), so it can be bound to a target variable in
the with/as form.

When the with block terminates, the \_\_exit\_\_ method:
1. Checks an exception was passed as exc_type; if so, gen.throw(exception) is in‐
voked, causing the exception to be raised in the yield line inside the generator
function body.
2. Otherwise, next(gen) is called, resuming the execution of the generator function
body after the yield.

In [0]:
# Example 15-7. mirror_gen_exc.py: generator-based context manager implementing exception handling—same external behavior as Example 15-3
import contextlib
@contextlib.contextmanager
def looking_glass():
  import sys
  original_write = sys.stdout.write
  
  def reverse_write(text):             
    original_write(text[::-1])
    
  sys.stdout.write = reverse_write
  msg = ''
  try:
    yield 'JABBERWOCKY'
  except ZeroDivisionError:
    msg = 'Please DO NOT divide by zero!'
  finally:
    sys.stdout.write = original_write
    if msg:
      print(msg)

## Further Reading
Chapter 8, “Compound Statements,” in The Python Language Reference says pretty much
everything there is to say about else clauses in if, for, while, and try statements.
Regarding Pythonic usage of try/except, with or without else, Raymond Hettinger
has a brilliant answer to the question “Is it a good practice to use try-except-else in
Python?” in StackOverflow. Alex Martelli’s Python in a Nutshell, 2E (O’Reilly), has a
chapter about exceptions with an excellent discussion of the EAFP style, crediting com‐
puting pioneer Grace Hopper for coining the phrase “It’s easier to ask forgiveness than
permission".